# Napari image viewer

Uses the octopuslite dask image loader to automatically sort and load images from multichannel time lapse experiments. Can also use btrack to load associated tracks and segmentation images.

In [1]:
import napari
from octopuslite import DaskOctopusLiteLoader

In [6]:
import numpy

# testing transform implementation

In [ ]:
import os

expt = 'ND0010'
pos = 'Pos0'
image_path = f'/home/nathan/data/kraken/ras/{expt}/{pos}/{pos}_images'
transform_path = f'/home/nathan/data/kraken/ras/{expt}/{pos}/gfp_transform_tensor_14.npy'
if os.path.exists(image_path):
    print(expt, pos)
#     trans = numpy.load(transform_path)
#     if len(trans.shape) == 4:
#         print('reshape')
#         numpy.save(transform_path, numpy.reshape(trans, (len(trans)*len(trans[0]), 3, 3)))
    images = DaskOctopusLiteLoader(image_path, remove_background=False, 
                                   transforms=transform_path,
                                   crop = (1200, 1600))
    gfp = images['gfp']
#     rfp = images['rfp']
#     irfp = images['irfp']
    masks = images['mask'].astype(int)
    viewer = napari.Viewer()
    viewer.add_image(gfp,
                     contrast_limits=[0,255],
                     blending = 'additive', 
                     colormap = 'green',
                    name = f'transform_{14}')#, is_pyramid=False)
#     viewer.add_image(rfp, name="rfp", 
#                      contrast_limits=[0,55], 
#                      blending = 'additive', colormap = 'magenta')#, is_pyramid=False)
#     viewer.add_image(irfp, name="irfp", 
#          contrast_limits=[0,255], 
#          blending = 'additive', colormap = 'cyan')#
    viewer.add_labels(masks, name = f'trans{expt}/{pos}',
                   )
    viewer.show(block=True)

ND0010 Pos0
Using cropping: (1200, 1600)


In [12]:
trans

array([[[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]],

       [[ 1.00000000e+00,  0.00000000e+00, -2.23294164e+02],
        [ 0.00000000e+00,  1.00000000e+00,  8.28624883e+01],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]],

       [[ 1.00000000e+00,  0.00000000e+00, -6.60374175e+01],
        [ 0.00000000e+00,  1.00000000e+00, -1.08065656e+02],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]],

       ...,

       [[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]],

       [[ 1.00000000e+00,  0.00000000e+00, -2.84351028e+00],
        [ 0.00000000e+00,  1.00000000e+00, -1.21357058e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]],

       [[ 1.00000000e+00,  0.00000000e+00, -1.81824139e+00],
 

# Load images

In [ ]:
# define experiment ID and select a position
expt = 'ND0010'
pos = 'Pos0'
# point to where the data is
root_dir = '/home/nathan/data/kraken/ras'
image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
transform_path = f'{root_dir}/{expt}/{pos}/gfp_transform_tensor.npy'
tracks_path = f'{root_dir}/{expt}/{pos}/tracks.h5'
# lazily load images
images = DaskOctopusLiteLoader(image_path, 
                               transforms = transform_path,
                               crop = (1200,1600),
                               remove_background = False)
images.channels

In [ ]:
gfp = images['gfp']
rfp = images['rfp']
irfp = images['irfp']
masks = images['mask']

In [ ]:
import numpy as np
for i in rfp[1109:1113]:
    print(np.mean(i.compute()))

### Add tracks

In [ ]:
import btrack
from btrack.utils import tracks_to_napari

#### iRFP tracks

In [ ]:
with btrack.dataio.HDF5FileHandler(tracks_path, 
                                   'r', obj_type='obj_type_1'
                                  ) as h:
    irfp_tracks = h.tracks

In [ ]:
irfp_tracks, irfp_properties, irfp_graph = tracks_to_napari(irfp_tracks, ndim = 2)

#### RFP tracks

In [ ]:
with btrack.dataio.HDF5FileHandler(tracks_path, 
                                   'r', obj_type='obj_type_2'
                                  ) as h:
    rfp_tracks = h.tracks

In [ ]:
rfp_tracks, rfp_properties, rfp_graph = tracks_to_napari(rfp_tracks, ndim = 2)

# Launch viewer

In [ ]:
viewer = napari.Viewer()
viewer.add_image(gfp, name="gfp", 
                 contrast_limits=[0,255],
                 blending = 'additive', colormap = 'green')#, is_pyramid=False)
viewer.add_image(rfp, name="rfp", 
                 contrast_limits=[0,55], 
                 blending = 'additive', colormap = 'magenta')#, is_pyramid=False)
viewer.add_image(irfp, name="irfp", 
                 contrast_limits=[0,255], 
                 blending = 'additive', colormap = 'cyan')#
viewer.add_image(masks, name = 'masks',
                contrast_limits=[0,255],
               blending = 'additive',)
viewer.add_tracks(irfp_tracks)
viewer.add_tracks(rfp_tracks)